09.2
10/31

In [21]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [22]:
# Load the datasets using Pandas
weather_data = pd.read_csv("weather_historic_data.csv")
station_data = pd.read_csv("station_historic_availability_data.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'weather_historic_data.csv'

In [4]:
weather_data.dtypes

Unnamed: 0       int64
time            object
temperature    float64
windspeed      float64
pressure       float64
description     object
cloudiness       int64
dtype: object

In [5]:
weather_data

,Unnamed: 0,time,temperature,windspeed,pressure,description,cloudiness
0,0,2023-03-03 21:38:52,5.20,2.06,1033.0,light intensity drizzle,75
1,1,2023-03-03 22:10:54,4.78,2.06,1033.0,broken clouds,75
2,2,2023-03-03 22:32:40,4.93,3.60,1033.0,broken clouds,75
3,3,2023-03-03 23:07:44,4.98,3.60,1033.0,broken clouds,75
4,4,2023-03-03 23:37:27,5.14,2.68,1033.0,broken clouds,75
...,...,...,...,...,...,...,...
1316,1316,2023-03-31 07:35:42,9.59,3.60,986.0,mist,75
1317,1317,2023-03-31 08:06:16,9.59,3.60,987.0,broken clouds,75
1318,1318,2023-03-31 08:45:17,9.78,3.60,988.0,light intensity drizzle,75
1319,1319,2023-03-31 09:08:07,9.78,3.60,989.0,light intensity drizzle,75


In [6]:
weather_data = weather_data.drop(columns=['Unnamed: 0'])
station_data = station_data.drop(columns=['Unnamed: 0'])

In [7]:
# Preprocess the data
weather_data['time'] = pd.to_datetime(weather_data['time'])
station_data['last_update'] = pd.to_datetime(station_data['last_update'])

In [8]:
# Extract day and hour
weather_data['year'] = weather_data['time'].dt.year
weather_data['month'] = weather_data['time'].dt.month
weather_data['day'] = weather_data['time'].dt.day
weather_data['hour'] = weather_data['time'].dt.hour
weather_data['minute'] = weather_data['time'].dt.minute
weather_data['is_weekday'] = ((weather_data['time'].dt.weekday >= 0) & (weather_data['time'].dt.weekday <= 4)).astype(int)


station_data['year'] = station_data['last_update'].dt.year
station_data['month'] = station_data['last_update'].dt.month
station_data['day'] = station_data['last_update'].dt.day
station_data['hour'] = station_data['last_update'].dt.hour
station_data['minute'] = station_data['last_update'].dt.minute

In [9]:
# One-hot encode the 'description' column
weather_data = pd.get_dummies(weather_data, columns= ['description'])

In [10]:
# Merge the datasets
merged_data = pd.merge(station_data, weather_data, on = ['year','month','day','hour','minute'])

In [11]:
# Drop every redundent columns
merged_data = merged_data.drop(columns=['last_update', 'time'])

In [12]:
merged_data

,number,available_bikes,available_bike_stands,year,month,day,hour,minute,temperature,windspeed,...,description_light intensity shower rain,description_light rain,description_light snow,description_mist,description_moderate rain,description_overcast clouds,description_scattered clouds,description_shower rain,description_sleet,description_snow
0,30,9,11,2023,3,3,21,38,5.20,2.06,...,0,0,0,0,0,0,0,0,0,0
1,31,13,7,2023,3,3,21,38,5.20,2.06,...,0,0,0,0,0,0,0,0,0,0
2,98,7,33,2023,3,3,21,38,5.20,2.06,...,0,0,0,0,0,0,0,0,0,0
3,60,23,7,2023,3,3,21,38,5.20,2.06,...,0,0,0,0,0,0,0,0,0,0
4,50,17,23,2023,3,3,21,38,5.20,2.06,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29937,79,12,15,2023,3,31,9,43,9.86,2.57,...,0,0,0,1,0,0,0,0,0,0
29938,69,24,16,2023,3,31,9,43,9.86,2.57,...,0,0,0,1,0,0,0,0,0,0
29939,85,1,34,2023,3,31,9,43,9.86,2.57,...,0,0,0,1,0,0,0,0,0,0
29940,57,4,19,2023,3,31,9,43,9.86,2.57,...,0,0,0,1,0,0,0,0,0,0


In [13]:
# Engineer is_busy_hours feature
merged_data['is_busy_hours'] = ((merged_data['hour'] >= 7) & (merged_data['hour'] <= 10)) | ((merged_data['hour'] >= 16) & (merged_data['hour'] <= 19)).astype(int)

In [14]:
merged_data['cold_weather'] = (merged_data['temperature'] < 5).astype(float)
merged_data['windy_weather'] = (merged_data['windspeed'] > 8).astype(float)

In [15]:
merged_data.head(20).to_csv('first_20_rows.csv', index=False)

In [16]:
unique_stations = list(merged_data['number'].unique())
unique_stations.sort()

In [17]:
for station_id in unique_stations:
    station_data = merged_data[merged_data['number'] == station_id]
    
    # Split the data into training and testing sets
    X = station_data.drop(columns=['available_bikes', 'available_bike_stands'])
    y = station_data[['available_bikes', 'available_bike_stands']]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train a machine learning model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Serialize the trained model into a file called model.pkl
    with open(f'model_{station_id}.pkl', 'wb') as handle:
        pickle.dump(model, handle, pickle.HIGHEST_PROTOCOL)

    # Deserialize the model.pkl file into an object called model
    with open(f'model_{station_id}.pkl', 'rb') as handle:
        model = pickle.load(handle)

    # Evaluate the model
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f"Mean Squared Error for station {station_id}:", mse)

Mean Squared Error for station 1: 59.35736812050393
Mean Squared Error for station 2: 9.507934829413863
Mean Squared Error for station 3: 26.796942896744277
Mean Squared Error for station 4: 38.52748938749414
Mean Squared Error for station 5: 105.38682146139068
Mean Squared Error for station 6: 33.31144223189794
Mean Squared Error for station 7: 54.81519736393713
Mean Squared Error for station 8: 53.110093590903006
Mean Squared Error for station 9: 45.86659379763565
Mean Squared Error for station 10: 20.277949072055705
Mean Squared Error for station 11: 29.091532486207313
Mean Squared Error for station 12: 15.889494401798142
Mean Squared Error for station 13: 27.810486450210824
Mean Squared Error for station 14: 66.31381413881059
Mean Squared Error for station 15: 6.00681691065445
Mean Squared Error for station 16: 20.5272548318641
Mean Squared Error for station 17: 18.00827582059606
Mean Squared Error for station 18: 28.394062211386633
Mean Squared Error for station 19: 86.14554935055

In [18]:
print(X.columns)

Index(['number', 'year', 'month', 'day', 'hour', 'minute', 'temperature',
       'windspeed', 'pressure', 'cloudiness', 'is_weekday',
       'description_broken clouds', 'description_clear sky',
       'description_few clouds', 'description_fog',
       'description_heavy intensity rain',
       'description_light intensity drizzle',
       'description_light intensity drizzle rain',
       'description_light intensity shower rain', 'description_light rain',
       'description_light snow', 'description_mist',
       'description_moderate rain', 'description_overcast clouds',
       'description_scattered clouds', 'description_shower rain',
       'description_sleet', 'description_snow', 'is_busy_hours',
       'cold_weather', 'windy_weather'],
      dtype='object')


This code is training a separate Linear Regression model for each bike station and predicting the number of available bikes (available_bikes) 
and available bike stands (available_bike_stands) for each station.
The predictions are made using the testing data (X_test), which contains information about the weather, time, and bike station details. After making predictions, the code calculates the mean squared error (MSE) for each station's model to evaluate its performance. The MSE is a measure of how well the model's predictions match the true values from the testing data.

In [19]:
def get_future_weather_data(date):
    # Replace this with actual weather prediction API
    
    weather_data = {
        'temperature': 12,
        'windspeed': 5,
        'pressure': 1015,
        'cloudiness': 20,
        'description': 'broken clouds',
    }
    
    # Convert the dictionary to a DataFrame
    future_weather_data = pd.DataFrame(weather_data, index=[0])
    
    # One-hot encode the 'description' column
    future_weather_data = pd.get_dummies(future_weather_data, columns=['description'])
    
    return future_weather_data

In [20]:
def predict_for_future_date(station_id, future_date):
    # Load the model
    with open(f'model_{station_id}.pkl', 'rb') as handle:
        model = pickle.load(handle)

    # Get future weather data
    future_weather_data = get_future_weather_data(future_date)

    # Preprocess future_weather_data and make the prediction
    future_date = pd.to_datetime(future_date)
    day = future_date.day
    hour = future_date.hour
    year = future_date.year
    month = future_date.month
    is_weekday = int((future_date.weekday() >= 0) & (future_date.weekday() <= 4))
    is_busy_hours = int(((hour >= 7) & (hour <= 10)) | ((hour >= 16) & (hour <= 19)))
    cold_weather = (future_weather_data['temperature'][0] < 5).astype(float)
    windy_weather = (future_weather_data['windspeed'][0] > 8).astype(float)

    # Create input DataFrame
    input_df = future_weather_data.copy()
    input_df['year'] = year
    input_df['month'] = month
    input_df['day'] = day
    input_df['hour'] = hour
    input_df['number'] = station_id
    input_df['is_weekday'] = is_weekday
    input_df['is_busy_hours'] = is_busy_hours
    input_df['cold_weather'] = cold_weather
    input_df['windy_weather'] = windy_weather

    # Make sure the input DataFrame has the same columns as the training data
    for col in X.columns:
        if col not in input_df.columns:
            input_df[col] = 0

    # Reorder the columns to match the training data
    input_df = input_df[X.columns]

     # Make the prediction
    prediction = model.predict(input_df)

    available_bikes, available_bike_stands = prediction[0]

    # Ensure available_bikes is greater than 0
    available_bikes = max(0, available_bikes)

    # Ensure available_bike_stands is greater than 0 and less than or equal to 114
    available_bike_stands = max(0, min(available_bike_stands, 114))

    return int(available_bikes), int(available_bike_stands)

In [105]:
station_id = 42
future_date = "2023-04-01 13:00:00"
available_bikes, available_bike_stands = predict_for_future_date(station_id, future_date)
print("Predicted available bikes:", available_bikes)
print("Predicted available bike stands:", available_bike_stands)

Predicted available bikes: 21
Predicted available bike stands: 8
